In [9]:
import sys
sys.path.append('../')
import os
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from tqdm.notebook import trange
from instances.instance import *
from GeCo_Instance_generators.knapsack import  yang_instance
from GeCo_Instance_generators.packing import tang_instance_packing

In [10]:
generator = StochasticBinPackerGenerator()
problem = generator.generate_normal_instance(100,100,100,200)

-1
-1


In [11]:
MP = gp.Model("MP")
x = MP.addMVar((problem.s1_n_var,), name = "x")
#y = EP.addMVar((problem.s2_n_var, problem.k), name = "y")
MP.modelSense = problem.s1_direction
if problem.s2_direction == problem.s1_direction:
    MP.setObjective(
        problem.c @ x
    )
else:
    MP.setObjective(
        problem.c @ x
    )

MP.addConstr(
    problem.A @ x <= problem.b
)
MP.optimize()
mp_x = x.x

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 100 rows, 100 columns and 8009 nonzeros
Model fingerprint: 0xf86c31e0
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+02, 1e+03]
Presolve time: 0.01s
Presolved: 100 rows, 100 columns, 8009 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3950000e+32   5.001250e+33   2.395000e+02      0s
      47    3.4169744e+03   0.000000e+00   0.000000e+00      0s

Solved in 47 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.416974431e+03


In [12]:
SP = gp.Model("SP")
A = problem.other_info['original_A2']
b = problem.other_info['original_b2']
c = problem.other_info['original_c2']

y = SP.addMVar((A.shape[1], ), obj = c)
SP.modelSense = problem.s2_direction

SP.addConstr(
    A @ y <= b
)
SP.optimize()
y.x

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 100 rows, 100 columns and 7934 nonzeros
Model fingerprint: 0xbf71384c
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+02, 1e+03]
Presolve time: 0.01s
Presolved: 100 rows, 100 columns, 7934 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1975000e+32   4.954750e+33   1.197500e+02      0s
      65    1.6938286e+03   0.000000e+00   0.000000e+00      0s

Solved in 65 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.693828611e+03


array([ 0.        ,  0.        , 17.8132711 , 10.40485929,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , 10.75260629,  0.        ,  4.02975994,  8.53678109,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , 16.16695327,  0.        ,  0.        ,  0.        ,
        8.36115618, 13.99069323,  0.        ,  0.        , 20.80069043,
       11.31845042,  0.        ,  0.        , 10.11038755,  0.        ,
        0.        , 60.54145515,  0.        ,  0.        ,  8.59273713,
        0.        ,  0.        ,  0.        ,  0.        ,  7.251693  ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  9.46752861, 13.19671338,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  3.48863129,  0.        ,  0.        ,  0.        ,
        0.        , 43.88870571,  0.        ,  0.        , 10.15

In [13]:
EP = gp.Model("EP")
x = EP.addMVar((problem.s1_n_var,), name = "x")
y = EP.addMVar((problem.s2_n_var, problem.k), name = "y")
EP.modelSense = problem.s1_direction
if problem.s2_direction == problem.s1_direction:
    EP.setObjective(
        problem.c @ x + gp.quicksum(1 / problem.k * problem.q_list[s] @ y[:,s] for s in range(problem.k))
    )
else:
    EP.setObjective(
        problem.c @ x - gp.quicksum(1 / problem.k * problem.q_list[s] @ y[:,s] for s in range(problem.k))
    )

EP.addConstr(
    problem.A @ x <= problem.b
)

EP.addConstrs((
    problem.W_list[s] @ y[:,s] + problem.T_list[s] @ x <= problem.h_list[s] for s in range(problem.k)
))

EP.optimize()
ep_x = x.x

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20100 rows, 20100 columns and 3594809 nonzeros
Model fingerprint: 0xcf60b78d
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-03, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+02, 1e+03]
Presolve time: 1.34s
Presolved: 20100 rows, 20100 columns, 3594809 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.12s

Barrier statistics:
 Dense cols : 100
 AA' NZ     : 2.998e+06
 Factor NZ  : 3.023e+06 (roughly 40 MB of memory)
 Factor Ops : 4.707e+08 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Ti

In [14]:
ep_x == mp_x

array([ True,  True, False, False,  True, False,  True,  True, False,
       False,  True,  True,  True, False, False, False,  True,  True,
        True,  True, False, False,  True,  True, False, False,  True,
       False,  True, False, False,  True, False, False,  True, False,
       False, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False, False,  True,  True, False, False, False,
       False,  True,  True,  True, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False,  True, False,  True,  True, False,  True, False,
        True, False,  True, False,  True,  True,  True, False, False,
        True, False, False, False, False, False, False, False, False,
       False])

In [15]:
ep_x

array([ 0.        ,  0.        , 16.58388605, 12.08525027,  0.        ,
        0.85770904,  0.        ,  0.        , 10.12063262, 15.19654763,
        0.        ,  0.        ,  0.        , 21.22128449, 13.0427869 ,
        7.67704809,  0.        ,  0.        ,  0.        ,  0.        ,
        6.97037443,  5.47832869,  0.        ,  0.        ,  0.16418964,
       14.58400958,  0.        ,  0.        ,  0.        ,  7.8279119 ,
        0.20637298,  0.        ,  7.60548843, 26.77200063,  0.        ,
       15.93022704, 16.26244457,  2.55072385,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       10.88523206,  0.        ,  6.20967861,  2.16607057,  0.        ,
        0.        , 17.12459418,  9.81732868, 10.84902553,  4.56825175,
        0.        ,  0.        ,  0.        , 13.81492809,  0.        ,
        6.4703608 , 13.14856927, 18.68165534,  4.47169314,  3.8520323 ,
        0.        ,  0.        ,  2.80232384,  6.13070043,  2.94

problems are defined by: 
c, A, B (first stage jazz)

q_s, W_s, h_s, T_s, and clustering variables

In [16]:
MP = gp.Model("MP")
MP.Params.outputFlag = 0
x = MP.addMVar((problem.s1_n_var,), name = "x")
eta = MP.addMVar((problem.k,), name = "eta", ub = problem.eta_bounds[1], lb = problem.eta_bounds[0])

if problem.s1_direction == GRB.MAXIMIZE:
    MP.modelSense = GRB.MAXIMIZE
else:
    MP.modelSense = GRB.MINIMIZE

MP.setObjective(
    problem.c @ x + np.array([1/problem.k] * problem.k) @ eta 
)

c1 = MP.addConstr(
    problem.A @ x <= problem.b
)


cut_found = True
n_iters = 0
try: 
    while cut_found:
        cut_found = False
        q_vals = []
        if n_iters < 200:
            n_iters = n_iters + 1
            print(f"On Iteration: {n_iters}")
        else:
            break
        MP.update()
        MP.optimize()
        LB = 0
        x_i = x.x
        eta_i = eta.x
        for s in range(problem.k):
            SP = gp.Model("SP")
            SP.Params.outputFlag = 0  # turn off output
            SP.Params.method = 1      # dual simplex
            y = SP.addMVar((problem.s2_n_var,), name = "y", obj = problem.q_list[s])
            if problem.s2_direction == GRB.MAXIMIZE:
                SP.modelSense = GRB.MAXIMIZE
            else:
                SP.modelSense = GRB.MINIMIZE
            res = SP.addConstr(
                problem.W_list[s] @ y <= problem.h_list[s] - (problem.T_list[s] @ x_i)
            )
            SP.optimize()
            Q = SP.ObjVal
            q_vals.append(Q)
            pi = res.Pi
            LB = LB + Q / problem.k
            if np.abs(Q - eta_i[s]) > 0.00001:
                if Q < eta_i[s]:
                    cut_found = True
                    p1 = pi @ problem.h_list[s]
                    p2 = pi @ problem.T_list[s]
                    if problem.s2_direction == GRB.MAXIMIZE:
                        MP.addConstr(
                            eta[s] <= p1 - gp.quicksum(p2[a] * x[a] for a in range(problem.s1_n_var))
                        )
                    else:
                        MP.addConstr(
                            eta[s] >= p1 - gp.quicksum(p2[a] * x[a] for a in range(problem.s1_n_var))
                        )        
        print(f"Iteration {n_iters}: LB = {LB}. UB = {MP.ObjVal}")
except:
    print(f"Errored out on iteration: {n_iters} scenario {s}")



On Iteration: 1
Iteration 1: LB = 14048.286079704556. UB = 123166.97443122778
On Iteration: 2
Iteration 2: LB = 15988.116562499023. UB = 19344.19631656641
On Iteration: 3
Iteration 3: LB = 15781.143047629988. UB = 18906.171159770398
On Iteration: 4
Iteration 4: LB = 15880.707396929376. UB = 18788.98595642744
On Iteration: 5
Iteration 5: LB = 15898.265301556. UB = 18759.49153003408
On Iteration: 6
Iteration 6: LB = 15870.210961372562. UB = 18748.278543747863
On Iteration: 7
Iteration 7: LB = 15905.855590777008. UB = 18743.980758174235
On Iteration: 8
Iteration 8: LB = 15891.832038311597. UB = 18742.70998203423
On Iteration: 9
Iteration 9: LB = 15894.004152360289. UB = 18742.28693148061
On Iteration: 10
Iteration 10: LB = 15891.435129247533. UB = 18742.20474885221
On Iteration: 11
Iteration 11: LB = 15892.398404575999. UB = 18742.1916217569
On Iteration: 12
Iteration 12: LB = 15892.256469848804. UB = 18742.191094389374
On Iteration: 13
Iteration 13: LB = 15892.11064909395. UB = 18742.191